In [1]:
import os
import re
import pandas as pd
import pickle
import collections
import numpy as np
import math
import ast
from time import gmtime, strftime
import re
import time
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [2]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

/Users/kimjisoo/opt/anaconda3/envs/preprocessing/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (36,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
gf_raw_fl_jver = pd.read_csv("gf_raw_fl_jver.csv")
gf_raw_fl_jver = gf_raw_fl_jver.iloc[:, 1:]

In [4]:
for _ in range(len(gf_raw_fl_jver)):
    gf_raw_fl_jver.at[_, 'product_info'] = eval(gf_raw_fl_jver.loc[_, 'product_info'])

In [31]:
column_name = ['브랜드명', '제품명','제품 id', '리뷰', '별점', '리뷰 일자', '사용자 리뷰 개수', '사용자 피부 타입', '사용자 나이', '사용자 성별', '사용자 이름']
df_review = pd.DataFrame(columns = column_name)
exception = []

In [32]:
idx = 0

for i in tqdm(list(gf_raw_fl_jver[40000:50000].index)):
    prd_info = list(gf_raw_fl_jver.loc[i, 'product_info'].items())
    prd_id = int(prd_info[2][-1])

    if prd_id in list(v4['제품 id']):
        try:
            prd_info = list(gf_raw_fl_jver.loc[i, 'product_info'].items())
            brd_name = prd_info[0][-1]
            prd_id = prd_info[2][-1]
            prd_name = prd_info[3][-1]
            review_lst = prd_info[30][-1] #모든 리뷰 리스트
            review_date_lst = prd_info[31][-1] #모든 리뷰 일자 리스트
            rating_lst = prd_info[32][-1] #모든 별점 리스트
            user_review_cnt_lst = prd_info[33][-1] #모든 사용자 리뷰 개수 리스트
            user_info_lst = prd_info[34][-1] #모든 사용자 정보 리스트
            user_name_lst = prd_info[35][-1] # 모든 사용자 이름 리스트

            j2 = 0
            for j in range(len(review_lst)): 
                review = review_lst[j]
                review_date = review_date_lst[j]
                rating = rating_lst[j]
                user_review_cnt = user_review_cnt_lst[j].split(' ')[-1]
                user_age = user_info_lst[j2]
                user_skin_type = user_info_lst[j2+1]
                user_sex = user_info_lst[j2+2]
                user_info = user_info_lst[j]
                user_name = user_name_lst[j]
                j2 += 3

                review_info = {
                    "브랜드명": brd_name,
                    "제품명": prd_name,
                    "제품 id": prd_id,
                    "리뷰": review,
                    "별점": rating,
                    "리뷰 일자": review_date,
                    "사용자 리뷰 개수": user_review_cnt,
                    "사용자 나이": user_age,
                    "사용자 피부 타입": user_skin_type,
                    "사용자 성별": user_sex,
                    "사용자 이름": user_name

                }

                df_review.loc[idx] = review_info
                idx += 1

        except:
            exception.append(i)
            pass

  0%|          | 0/10000 [00:00<?, ?it/s]

In [33]:
exception

[]

In [34]:
df_review

,브랜드명,제품명,제품 id,리뷰,별점,리뷰 일자,사용자 리뷰 개수,사용자 피부 타입,사용자 나이,사용자 성별,사용자 이름
0,에네스티,수안수 실크스킨 풋크림,123228,너무 가성비좋은 풋크림저렴한데 바르구나면 참촉촉,5,1개월전,20,건성,28세,여,니키콩자반
1,에네스티,수안수 실크스킨 풋크림,123228,발이 많이 보들보들 해졌어요. 더 좋은 풋크림이 있을것 같기도 하지만 꽤 괜찮아서 ...,4,2개월전,77,건성,27세,여,풋사과젤리
2,에네스티,수안수 실크스킨 풋크림,123228,뉴트로지나만 사용하다가 언니네집에 놀러갔는데 이 제품 있어서 사용해봤어요! 풋크림은...,4,4개월전,990,복합성,38세,여,vivistar
3,에네스티,수안수 실크스킨 풋크림,123228,가격도 저렴하고 무엇보다 보습력이 아주 좋아요! 지금까지 풋크림을 따로 써야하는 이...,5,2021.01.23,192,민감성,22세,여,Yellowston
4,에네스티,수안수 실크스킨 풋크림,123228,인터넷에서 2개 5천원대에 구매해서 쓰고있어요.겨울마다 발에 한포진같이 간지러우면서...,3,2021.01.13,118,복합성,25세,여,yoo5
...,...,...,...,...,...,...,...,...,...,...,...
123472,이니스프리,[단종] 슈퍼푸드 브로콜리 클리어링 토너,53841,지성피부용이라서 엄청 가볍고 화한느낌 날줄알았는데생각보다 수분감도 있고 괜찮았어요!...,5,2017.10.17,192,건성,29세,여,쀼찌뺘찌
123473,이니스프리,[단종] 슈퍼푸드 브로콜리 클리어링 토너,53841,두통 썼습니다. 미친듯한 효과는 없지만 무난하고 얼굴을 개운하게 정리해줘요. 이걸 ...,5,2017.10.02,10,민감성,34세,여,최미미쨩
123474,이니스프리,[단종] 슈퍼푸드 브로콜리 클리어링 토너,53841,블랙헤드 생산중인 언니들 !!! 제발! 이거! 쓰세요!한달째 쓰고있는데 블랙헤드 몇...,5,2017.09.26,104,민감성,24세,여,손소소
123475,이니스프리,[단종] 슈퍼푸드 브로콜리 클리어링 토너,53841,한통 완전히 비우고 리뷰 씁니다무겁지 않은 물토너입니다가볍게 쓰기 좋구요알콜향이 조...,3,2017.09.26,499,복합성,25세,여,쮸잇쮸잇츄우


In [35]:
df_review.to_csv("df_v4_review_40000_50000.csv")

In [38]:
df_v4_review_0_10000 = pd.read_csv("df_v4_review_0_10000.csv")
df_v4_review_0_10000 = df_v4_review_0_10000.iloc[:, 1:]

df_v4_review_10000_20000 = pd.read_csv("df_v4_review_10000_20000.csv")
df_v4_review_10000_20000 = df_v4_review_10000_20000.iloc[:, 1:]

df_v4_review_20000_30000 = pd.read_csv("df_v4_review_20000_30000.csv")
df_v4_review_20000_30000 = df_v4_review_20000_30000.iloc[:, 1:]

df_v4_review_30000_40000 = pd.read_csv("df_v4_review_30000_40000.csv")
df_v4_review_30000_40000 = df_v4_review_30000_40000.iloc[:, 1:]

df_v4_review_40000_50000 = pd.read_csv("df_v4_review_40000_50000.csv")
df_v4_review_40000_50000 = df_v4_review_40000_50000.iloc[:, 1:]

df_v4_review_50000_60000 = pd.read_csv("df_v4_review_50000_60000.csv")
df_v4_review_50000_60000 = df_v4_review_50000_60000.iloc[:, 1:]

df_v4_review_60000_70000 = pd.read_csv("df_v4_review_60000_70000.csv")
df_v4_review_60000_70000 = df_v4_review_60000_70000.iloc[:, 1:]

df_v4_review_70000_80000 = pd.read_csv("df_v4_review_70000_80000.csv")
df_v4_review_70000_80000 = df_v4_review_70000_80000.iloc[:, 1:]

df_v4_review_80000_90000 = pd.read_csv("df_v4_review_80000_90000.csv")
df_v4_review_80000_90000 = df_v4_review_80000_90000.iloc[:, 1:]

df_v4_review_90000_98612 = pd.read_csv("df_v4_review_90000_98612.csv")
df_v4_review_90000_98612 = df_v4_review_90000_98612.iloc[:, 1:]

In [40]:
df_v4_review = pd.concat([df_v4_review_0_10000, df_v4_review_10000_20000, df_v4_review_20000_30000, 
                         df_v4_review_30000_40000, df_v4_review_40000_50000, df_v4_review_50000_60000,
                         df_v4_review_60000_70000, df_v4_review_70000_80000, df_v4_review_80000_90000,
                         df_v4_review_90000_98612])

In [50]:
df_v4_review = df_v4_review.reset_index(drop=True)

In [57]:
len(df_v4_review)

933555

In [56]:
df_v4_review.loc[df_v4_review.duplicated(['리뷰']) == True]

,브랜드명,제품명,제품 id,리뷰,별점,리뷰 일자,사용자 리뷰 개수,사용자 피부 타입,사용자 나이,사용자 성별,사용자 이름
699,스팀베이스,로즈허브 빌드 업 토너,112999,저는 피부가 악건성으로 민감하기도 한 피부입니다기존에 이니스프리 인치치드 라인 스킨...,4,2019.02.13,98,건성,26세,여,응딩디
703,스팀베이스,로즈허브 빌드 업 토너,112999,피부상태 안좋을때 다 내려놓고 실험하듯써본 토너..자극없고 순해서 트러블이 심해지지...,4,2018.12.11,257,중성,24세,여,또용토용
4804,미미로린스,"모먼트 플랜트 핸드크림 [헤이, 그린]",128692,여기 핸드워시 향이 너무 좋아서 핸드크림도 사게 됨. 핸드워시의 헤이 그린 향은 그...,3,2개월전,339,복합성,45세,여,카소봉
5746,라로슈포제,에빠끌라 클래리파잉 모이스춰라이징 로션,12208,아침 라로클렌징-라로로션-라로B5저녁 라로클렌징-토너팩-라로B5-센텔라가루도포지성피...,4,2021.01.31,145,복합성,24세,여,꼰도
5914,라로슈포제,에빠끌라 K 데일리 리뉴얼 에센스,12215,무조건 사세요ㅋㅋㅋㅋ햇빛알레르기 생긴후로 민감하기만 햇던 내피부를 되살려준 라로슈포...,5,2018.03.26,133,민감성,34세,여,jrang
...,...,...,...,...,...,...,...,...,...,...,...
933473,더바디샵,[단종] 드롭스 오브 유스 유스 리퀴드 필,94094,저는 이 제품 괜찮게 쓰고 있습니다. 건성이라 피부가 자주 떠서 필링을 자주 해 줘...,3,2018.05.04,213,복합성,22세,여,설라18
933474,더바디샵,[단종] 드롭스 오브 유스 유스 리퀴드 필,94094,제가 걸 왜 3만 2천원이나 주고 샀는지 모르겠네요. 필링 한답시고 쓰면 빨개지고 ...,1,2018.03.31,317,건성,17세,여,박페페
933475,더바디샵,[단종] 드롭스 오브 유스 유스 리퀴드 필,94094,세일해서 가격 25000원 정도 줬어요좋은점은 쉽게 쓸 수 있다는 점 짜서 문지르면...,3,2017.10.16,130,중성,33세,여,versacrum
933553,데이리쎄,더마클린 초초 필링 패드,128078,한통 다 써가서 후기남겨요. 우선 저자극이에요. 동봉된 내장집게로 한장씩 집어서 쓸...,3,2020.01.18,62,건성,35세,여,크리유


In [58]:
df_v4_review = df_v4_review.drop_duplicates(subset=['리뷰'])

In [59]:
len(df_v4_review)

727248

In [61]:
df_v4_review = df_v4_review.reset_index(drop=True)

In [62]:
df_v4_review.to_csv("df_v4_review.csv")

In [68]:
df_v4_review.loc[100:150]

,브랜드명,제품명,제품 id,리뷰,별점,리뷰 일자,사용자 리뷰 개수,사용자 피부 타입,사용자 나이,사용자 성별,사용자 이름
100,갸마르드,리치 모이스춰라이징 크림,27932,프랑스 몽쥬약국에서 판매원이 현존하는 가장 우수한 성분의 유기농 크림이라고 말했던 ...,4,2016.06.01,219,복합성,38세,여,크림케이크
101,갸마르드,리치 모이스춰라이징 크림,27932,개인적으로 유기농 화장품 치고 향이 제일 좋았던 제품얇게 발라도 충분하기 때문에 굉...,4,2015.12.31,67,건성,28세,여,Whatnee
102,갸마르드,워터,6115,유황향이 나긴하지만 아벤느미스트랑 같이 써보면 확실히 그냥 물은 아니구나라는걸 느낄...,4,2016.04.25,34,복합성,31세,여,유니돌프
103,갸마르드,워터,6115,회사에 두고 쓰고 있는 제품.100퍼 유기농에 전성분 온천수라는 위엄 넘치는 미스트...,4,2016.04.04,329,복합성,32세,여,파항
104,갸마르드,워터,6115,안개 분사가 되어서 좋았고온천수를 피부에 듬뿍 뿌려줄 수 있어서 좋았다꼭 다시 사고...,4,2015.12.31,67,건성,28세,여,Whatnee
105,갸마르드,워터,6115,얇은 화장솜에 적셔서 팩하면 열도내리고 정말 좋아요 ㅋㅋㅋ 특히 뾰루지 짜고 화끈거...,4,2015.08.12,200,복합성,33세,여,cumcok
106,갸마르드,워터,6115,1년간무지하게뿌렸던미스트ㅎ진짜한달에큰거한통씩쓸정도로즐겨썻음 가격도착한편이고해서썻는데...,3,2015.06.03,401,건성,34세,여,설구
107,갸마르드,모이스춰라이징 크림 마스크,6101,용량에 비해 가격이 좀 비싼 편이지만 비싼값합니다.. 일반 크림보다 적은양 사용해도...,5,2019.01.09,106,복합성,28세,여,잉쏭이
108,갸마르드,모이스춰라이징 크림 마스크,6101,샘플이 생겨서 썼다가 본품까지 구매함!!!!총평 : 향기좋음 +꽤매트하게발림+ 미온...,4,2017.11.29,187,건성,33세,여,홍시루
109,갸마르드,프로텍티브 샤워젤,27977,이거 징짜조아여...♥유기농이라 냄새는 별루지만 비정재라 좋은성분 그대로담겨있구 제...,5,2016.01.15,30,건성,26세,여,북극치킨


In [77]:
v4_prd_id_lst = list(set(df_v4_review['제품 id']))

In [80]:
len(set(df_v4_review['제품 id']))

44177

In [71]:
v4 = pd.read_csv("v4_update.csv")
v4 = v4.iloc[:, 1:]

/Users/kimjisoo/opt/anaconda3/envs/preprocessing/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (35,36,37,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [72]:
len(v4)

64216

In [78]:
v4_prd_no_review = [x for x in tqdm(v4['제품 id']) if x not in v4_prd_id_lst]

  0%|          | 0/64216 [00:00<?, ?it/s]

In [92]:
import pickle

with open("v4_prd_no_review.txt", 'wb') as f:
    pickle.dump(v4_prd_no_review, f)

In [83]:
v4_prd_no_review[:5]

[93849, 36978, 111421, 150371, 150374]

In [84]:
gf_raw_fl_jver_prd_id_lst = []

for i in tqdm(list(gf_raw_fl_jver.index)):
    prd_info = list(gf_raw_fl_jver.loc[i, 'product_info'].items())
    prd_id = int(prd_info[2][-1])
    gf_raw_fl_jver_prd_id_lst.append(prd_id)

  0%|          | 0/98612 [00:00<?, ?it/s]

In [87]:
gf_raw_fl_jver_prd_id_lst = list(set(gf_raw_fl_jver_prd_id_lst))

In [ ]:
import pickle

with open("lst_glowpick_crawling_link_v2.txt", 'rb') as f:
    lst_glowpick_crawling_link_v2 = pickle.load(f)

In [93]:
df_glowpick_prd_info_v2_211006 = pd.read_csv("df_glowpick_prd_info_v2_211006.csv")
df_glowpick_prd_info_v2_211006 = df_glowpick_prd_info_v2_211006.iloc[:, 1:]

df_glowpick_prd_info_v2_211007 = pd.read_csv("df_glowpick_prd_info_v2_211007.csv")
df_glowpick_prd_info_v2_211007 = df_glowpick_prd_info_v2_211007.iloc[:, 1:]

df_glowpick_prd_info_v2_211008 = pd.read_csv("df_glowpick_prd_info_v2_211008.csv")
df_glowpick_prd_info_v2_211008 = df_glowpick_prd_info_v2_211008.iloc[:, 1:]

In [94]:
df_glowpick_prd_info_v1 = pd.read_csv("df_glowpick_prd_info_v1.csv")
df_glowpick_prd_info_v1 = df_glowpick_prd_info_v1.iloc[:, 1:]

/Users/kimjisoo/opt/anaconda3/envs/preprocessing/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [95]:
df_glowpick_prd_info = pd.concat([df_glowpick_prd_info_v1, df_glowpick_prd_info_v2_211006,
                                 df_glowpick_prd_info_v2_211007, df_glowpick_prd_info_v2_211008])

In [97]:
len(set(df_glowpick_prd_info['제품 id']))

26628

In [ ]:
d